In [33]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from nltk import word_tokenize
from nltk import WordNetLemmatizer
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')
from nltk.corpus import wordnet
import string


columns=["txt", "l1-class", "l2-class", "l3-class"]
train_set = pd.read_csv("DBPEDIA_train.csv", header=None, names=columns)
test_set = pd.read_csv("DBPEDIA_test.csv", header=None, names=columns)
#Ograniczamy sie do klas l2
del train_set["l1-class"]
del train_set["l3-class"]
del test_set["l1-class"]
del test_set["l3-class"]
del columns[3]
del columns[1]

train_set = train_set.sample(frac=0.1)
test_set = test_set.sample(frac=0.1)

train_set_X = train_set["txt"]
train_set_Y = train_set["l2-class"]
test_set_X = test_set["txt"]
test_set_Y = test_set["l2-class"]

print(type(train_set_X))
print(train_set_X.shape)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\pioro\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\pioro\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\pioro\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


<class 'pandas.core.series.Series'>
(24094,)


In [37]:
train_set_X = train_set_X.apply(lambda text: text.translate(str.maketrans('', '', string.punctuation)))
test_set_X = test_set_X.apply(lambda text: text.translate(str.maketrans('', '', string.punctuation)))

train_set_X = train_set_X.apply(lambda text: text.lower())
test_set_X = test_set_X.apply(lambda text: text.lower())

print(train_set_X.iloc[4])

the darmstadt–worms railway is a standardgauge railway that is now partially closed it runs through southern hesse through the hessian ried hessische ried and so it is also called the riedbahn ried railway the section between darmstadt and riedstadtgoddelau is now largely closed the section between riedstadtgoddelau and biblis which is now considered part of the mannheim–frankfurt railway is of great importance for longdistance passenger services and rail freight traffic the last section from biblis to worms is used by regional passenger services and rail freight traffic


In [38]:
train_set_X = train_set_X.apply(lambda text: word_tokenize(text))
test_set_X = test_set_X.apply(lambda text: word_tokenize(text))

print(train_set_X.iloc[4])

['the', 'darmstadt–worms', 'railway', 'is', 'a', 'standardgauge', 'railway', 'that', 'is', 'now', 'partially', 'closed', 'it', 'runs', 'through', 'southern', 'hesse', 'through', 'the', 'hessian', 'ried', 'hessische', 'ried', 'and', 'so', 'it', 'is', 'also', 'called', 'the', 'riedbahn', 'ried', 'railway', 'the', 'section', 'between', 'darmstadt', 'and', 'riedstadtgoddelau', 'is', 'now', 'largely', 'closed', 'the', 'section', 'between', 'riedstadtgoddelau', 'and', 'biblis', 'which', 'is', 'now', 'considered', 'part', 'of', 'the', 'mannheim–frankfurt', 'railway', 'is', 'of', 'great', 'importance', 'for', 'longdistance', 'passenger', 'services', 'and', 'rail', 'freight', 'traffic', 'the', 'last', 'section', 'from', 'biblis', 'to', 'worms', 'is', 'used', 'by', 'regional', 'passenger', 'services', 'and', 'rail', 'freight', 'traffic']


In [39]:
from nltk.corpus import wordnet

lemmatizer = WordNetLemmatizer()
iterator = 0
def get_wordnet_pos(word):
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}

    return tag_dict.get(tag, wordnet.NOUN)

def list_lemmatize(list_of_words):
    new_list_of_words = []
    global iterator
    iterator += 1
    if iterator%10000 == 0:
        print(iterator)
    for word in list_of_words:
        new_list_of_words.append(lemmatizer.lemmatize(word, get_wordnet_pos(word)))
    return new_list_of_words

train_set_X = train_set_X.apply(list_lemmatize)
test_set_X = test_set_X.apply(list_lemmatize)

print(train_set_X.iloc[4])



10000
20000
30000
['the', 'darmstadt–worms', 'railway', 'be', 'a', 'standardgauge', 'railway', 'that', 'be', 'now', 'partially', 'close', 'it', 'run', 'through', 'southern', 'hesse', 'through', 'the', 'hessian', 'ried', 'hessische', 'ried', 'and', 'so', 'it', 'be', 'also', 'call', 'the', 'riedbahn', 'ried', 'railway', 'the', 'section', 'between', 'darmstadt', 'and', 'riedstadtgoddelau', 'be', 'now', 'largely', 'close', 'the', 'section', 'between', 'riedstadtgoddelau', 'and', 'biblis', 'which', 'be', 'now', 'consider', 'part', 'of', 'the', 'mannheim–frankfurt', 'railway', 'be', 'of', 'great', 'importance', 'for', 'longdistance', 'passenger', 'service', 'and', 'rail', 'freight', 'traffic', 'the', 'last', 'section', 'from', 'biblis', 'to', 'worm', 'be', 'use', 'by', 'regional', 'passenger', 'service', 'and', 'rail', 'freight', 'traffic']


In [41]:
from nltk.corpus import stopwords
stopWords = set(stopwords.words('english'))

def filter_stop_words(list_of_words):
    words_filtered = []
    for w in list_of_words:
        if w not in stopWords:
            words_filtered.append(w)
    return words_filtered

train_set_X = train_set_X.apply(filter_stop_words)
test_set_X = test_set_X.apply(filter_stop_words)

print(train_set_X.iloc[4])


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\pioro\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


['darmstadt–worms', 'railway', 'standardgauge', 'railway', 'partially', 'close', 'run', 'southern', 'hesse', 'hessian', 'ried', 'hessische', 'ried', 'also', 'call', 'riedbahn', 'ried', 'railway', 'section', 'darmstadt', 'riedstadtgoddelau', 'largely', 'close', 'section', 'riedstadtgoddelau', 'biblis', 'consider', 'part', 'mannheim–frankfurt', 'railway', 'great', 'importance', 'longdistance', 'passenger', 'service', 'rail', 'freight', 'traffic', 'last', 'section', 'biblis', 'worm', 'use', 'regional', 'passenger', 'service', 'rail', 'freight', 'traffic']


In [43]:
train_set_X = train_set_X.apply(lambda text: ' '.join(text))
test_set_X = test_set_X.apply(lambda text: ' '.join(text))

print(train_set_X.iloc[4])

darmstadt–worms railway standardgauge railway partially close run southern hesse hessian ried hessische ried also call riedbahn ried railway section darmstadt riedstadtgoddelau largely close section riedstadtgoddelau biblis consider part mannheim–frankfurt railway great importance longdistance passenger service rail freight traffic last section biblis worm use regional passenger service rail freight traffic


In [44]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(min_df=5, max_df=0.5)
train_features_set_X = vectorizer.fit_transform(train_set_X)
test_features_set_X = vectorizer.transform(test_set_X)
print(train_features_set_X.shape)


(24094, 17250)


In [50]:
from sklearn.naive_bayes import MultinomialNB
bayes = MultinomialNB()
bayes.fit(train_features_set_X, train_set_Y)
print(bayes.score(train_features_set_X, train_set_Y))
print(bayes.score(test_features_set_X, test_set_Y))
print(len(set(test_set_Y)))

0.9336764339669628
0.8657894736842106
70


In [59]:
index = 88
prediction = bayes.predict(test_features_set_X[index])
print(prediction[0] + str(test_set_Y.iloc[index] == prediction))
print(test_set_X.iloc[index])

Tournament[ True]
fa cup 1973–74 93rd season world old football knockout competition football association challenge cup fa cup short large number club enter tournament low english football league system meant competition start number preliminary qualify round 30 victorious team fourth round qualify progress first round proper


In [61]:
from sklearn import metrics
predictions = bayes.predict(test_features_set_X)
print(metrics.confusion_matrix(test_set_Y, predictions))
print(metrics.classification_report(test_set_Y, predictions))


[[ 14   0   0 ...   0   0   0]
 [  0  14   0 ...   0   0   0]
 [  0   0 368 ...   0   0   0]
 ...
 [  0   0   0 ...   3   0   0]
 [  0   0   0 ...   0   0   0]
 [  0   0   0 ...   0   0  20]]
                              precision    recall  f1-score   support

                       Actor       1.00      0.50      0.67        28
     AmusementParkAttraction       1.00      0.93      0.97        15
                      Animal       0.90      0.99      0.94       372
                      Artist       0.77      0.84      0.80       134
                     Athlete       0.86      0.96      0.91       800
                 BodyOfWater       0.93      0.91      0.92        43
                       Boxer       0.00      0.00      0.00         6
              BritishRoyalty       1.00      0.53      0.70        15
                 Broadcaster       0.94      0.97      0.95        93
                    Building       0.89      0.95      0.92       277
                     Cartoon       0.

E:\conda\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Maxing Macro avg precision w celu zapewnienia dobrej jakosci dla mniej popularnych klas
